In [1]:
import pandas as pd 
import seaborn as sns 
import numpy as np
import matplotlib.pyplot as plt

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import string

from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

In [48]:

data = pd.read_csv('C:/Users/Rhea/Documents/INNOVATION/PROJECT/Amazon_Phone_samsung_Reviews.csv',
                   names=['Phone_Name','Date','Rating_Out_of_5','Review_Title','Review_Text'])



In [49]:
data.head()

,Phone_Name,Date,Rating_Out_of_5,Review_Title,Review_Text
0,Phone_Name,Date,Rating_Out_of_5,Review_Title,Review_Text
1,Samsung Galaxy S7 G930F,"October 6, 2016",1.0,Manufacturer Defect,The Galaxy S7 was wonderful while it worked. G...
2,Samsung Galaxy S7 G930F,"August 27, 2016",1.0,False advertising,The description on this phone said Dual Sim an...
3,Samsung Galaxy S7 G930F,"November 10, 2016",5.0,overall I'm pretty satisfied. I was afraid I w...,"I bought this on June 29, 2016. So far this ph..."
4,Samsung Galaxy S7 G930F,"December 8, 2017",1.0,Was a fake phone!,If there was a 0 star of rate that. Recueved t...


In [4]:
data.shape

(820, 5)

In [5]:
data['Review_Text'].values[5]

'I bought this from seller "The Wireless Circle". I am a US AT&T customer in a 4G LTE market.First off, the phone came with the UK electrical outlet Fast Charger with an outlet adapter to US. This wasn\'t a problem to me other than bulkiness just not what I expected.Another thing, in the central midwest, I had a hard time being anywhere where I received good signal strength 4G LTE. I even called AT&T about it and attempted a new SIM card (thinking that possibly the card unlocked certain frequency bands). I ordinarily wouldn\'t care about this, except the battery statistics constantly say that the cell standby was using most of the battery.This weekend is when the real problem started. I was using the phone and when I looked back after a few seconds, the screen was frozen. I tried to end software but everything was unresponsive--the touch screen and the buttons. I tried restarting the phone: the power button produced no response. With older models, I would remove the battery at this poi

In [6]:
data_samples=data['Review_Text'].tolist()
data_samples[1]

"The description on this phone said Dual Sim and it wasn't.  I tried putting in a second SIM after buying one and was told the phone was for a single SIM.  That is false marketing.  I want myou money back or a DUAL Sim phone"

In [7]:
data_samples = [w.lower() for w in data_samples]

In [8]:
tokenized_docs = [word_tokenize(doc) for doc in data_samples]
print(tokenized_docs[5])

['i', 'bought', 'this', 'from', 'seller', '``', 'the', 'wireless', 'circle', "''", '.', 'i', 'am', 'a', 'us', 'at', '&', 't', 'customer', 'in', 'a', '4g', 'lte', 'market.first', 'off', ',', 'the', 'phone', 'came', 'with', 'the', 'uk', 'electrical', 'outlet', 'fast', 'charger', 'with', 'an', 'outlet', 'adapter', 'to', 'us', '.', 'this', 'was', "n't", 'a', 'problem', 'to', 'me', 'other', 'than', 'bulkiness', 'just', 'not', 'what', 'i', 'expected.another', 'thing', ',', 'in', 'the', 'central', 'midwest', ',', 'i', 'had', 'a', 'hard', 'time', 'being', 'anywhere', 'where', 'i', 'received', 'good', 'signal', 'strength', '4g', 'lte', '.', 'i', 'even', 'called', 'at', '&', 't', 'about', 'it', 'and', 'attempted', 'a', 'new', 'sim', 'card', '(', 'thinking', 'that', 'possibly', 'the', 'card', 'unlocked', 'certain', 'frequency', 'bands', ')', '.', 'i', 'ordinarily', 'would', "n't", 'care', 'about', 'this', ',', 'except', 'the', 'battery', 'statistics', 'constantly', 'say', 'that', 'the', 'cell', '

## Removing punctuation
Punctuation can help with tokenizers, but once you've done that, there's no reason to keep it around. There are tons of ways to remove punctuation. Since we have already learned regex, how would we do this?

In [9]:
print(string.punctuation)
str1=string.punctuation
str1=str1.replace('/','')
print(str1)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
!"#$%&'()*+,-.:;<=>?@[\]^_`{|}~


In [51]:
regex = re.compile('[%s]' % re.escape(str1)) #see documentation here: http://docs.python.org/2/library/string.html

tokenized_docs_no_punctuation = []

for review in tokenized_docs:
    
    new_review = []
    for token in review: 
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            new_review.append(new_token)
    
    tokenized_docs_no_punctuation.append(new_review)
    
print (tokenized_docs_no_punctuation[5])

['i', 'bought', 'this', 'from', 'seller', '  ', 'the', 'wireless', 'circle', '  ', ' ', 'i', 'am', 'a', 'us', 'at', ' ', 't', 'customer', 'in', 'a', '4g', 'lte', 'market first', 'off', ' ', 'the', 'phone', 'came', 'with', 'the', 'uk', 'electrical', 'outlet', 'fast', 'charger', 'with', 'an', 'outlet', 'adapter', 'to', 'us', ' ', 'this', 'was', 'n t', 'a', 'problem', 'to', 'me', 'other', 'than', 'bulkiness', 'just', 'not', 'what', 'i', 'expected another', 'thing', ' ', 'in', 'the', 'central', 'midwest', ' ', 'i', 'had', 'a', 'hard', 'time', 'being', 'anywhere', 'where', 'i', 'received', 'good', 'signal', 'strength', '4g', 'lte', ' ', 'i', 'even', 'called', 'at', ' ', 't', 'about', 'it', 'and', 'attempted', 'a', 'new', 'sim', 'card', ' ', 'thinking', 'that', 'possibly', 'the', 'card', 'unlocked', 'certain', 'frequency', 'bands', ' ', ' ', 'i', 'ordinarily', 'would', 'n t', 'care', 'about', 'this', ' ', 'except', 'the', 'battery', 'statistics', 'constantly', 'say', 'that', 'the', 'cell', '

## Cleaning text of stopwords
There are some really basic words that just don't matter. NLTK comes with a list of them for many languages.

In [52]:
tokenized_docs_no_stopwords = []
for doc in tokenized_docs_no_punctuation:
    new_term_vector = []
    for word in doc:
        if not word in stopwords.words('english'):
            new_term_vector.append(word)
    tokenized_docs_no_stopwords.append(new_term_vector)
            
print (tokenized_docs_no_stopwords[1])

['description', 'phone', 'said', 'dual', 'sim', 'n t', ' ', 'tried', 'putting', 'second', 'sim', 'buying', 'one', 'told', 'phone', 'single', 'sim', ' ', 'false', 'marketing', ' ', 'want', 'myou', 'money', 'back', 'dual', 'sim', 'phone']


### Stemming and Lemmatizing
If you have taken linguistics, you may be familiar with morphology. This is the belief that words have a root form. If you want to get to the basic term meaning of the word, you can try applying a stemmer or lemmatizer. Here are three very popular methods ready to go right out of the NLTK box. It's up to you to see which one you want to use.

In [53]:
porter = PorterStemmer()
snowball = SnowballStemmer('english')
wordnet = WordNetLemmatizer()

preprocessed_docs = []

for doc in tokenized_docs_no_stopwords:
    final_doc = []
    for word in doc:
        final_doc.append(wordnet.lemmatize(word))
        #final_doc.append(snowball.stem(word))
        #final_doc.append(wordnet.lemmatize(word)) #note that lemmatize() can also takes part of speech as an argument!
    preprocessed_docs.append(final_doc)

print (preprocessed_docs[1])

['description', 'phone', 'said', 'dual', 'sim', 'n t', ' ', 'tried', 'putting', 'second', 'sim', 'buying', 'one', 'told', 'phone', 'single', 'sim', ' ', 'false', 'marketing', ' ', 'want', 'myou', 'money', 'back', 'dual', 'sim', 'phone']


In [54]:
pos_tags=[]

for i in preprocessed_docs:
    pos_tags.append(nltk.pos_tag(i)) 

In [55]:
pos_tags[1]

[('description', 'NN'),
 ('phone', 'NN'),
 ('said', 'VBD'),
 ('dual', 'JJ'),
 ('sim', 'NN'),
 ('n t', 'NN'),
 (' ', 'NNP'),
 ('tried', 'VBD'),
 ('putting', 'VBG'),
 ('second', 'JJ'),
 ('sim', 'JJ'),
 ('buying', 'NN'),
 ('one', 'CD'),
 ('told', 'VBD'),
 ('phone', 'NN'),
 ('single', 'JJ'),
 ('sim', 'NN'),
 (' ', 'NN'),
 ('false', 'JJ'),
 ('marketing', 'NN'),
 (' ', 'NN'),
 ('want', 'VBP'),
 ('myou', 'NN'),
 ('money', 'NN'),
 ('back', 'RB'),
 ('dual', 'JJ'),
 ('sim', 'NN'),
 ('phone', 'NN')]

In [56]:
noun_words=[]
verb_words=[]
adj_words=[]
for doc in pos_tags:
    for word_pos in doc :
        if word_pos[1] in ('NN','NNS','NNP','NNPS'):
            noun_words.append(word_pos[0]) 
        elif word_pos[1] in ('VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'):
            verb_words.append(word_pos[0])
        elif word_pos[1] in ('JJR','JJS','JJ'):
            adj_words.append(word_pos[0])
        

In [57]:
print(len(set(adj_words)))

1044


In [58]:
import operator


In [59]:
noun_dict=dict((i, noun_words.count(i)) for i in noun_words)
noun_dict = sorted(noun_dict.items(), key=operator.itemgetter(1),reverse=True)
print((noun_dict))

[(' ', 3573), ('phone', 984), ('samsung', 187), ('work', 169), (' s', 148), ('s7', 144), ('camera', 128), ('battery', 119), ('time', 108), ('screen', 107), ('  ', 103), ('card', 91), ('galaxy', 90), ('problem', 90), ('   ', 88), ('month', 76), ('everything', 72), ('version', 71), ('day', 71), ('use', 69), ('issue', 68), ('product', 67), (' m', 65), ('n t', 62), ('thing', 57), ('device', 57), ('quality', 55), ('life', 49), ('case', 47), ('box', 46), ('feature', 46), ('water', 45), ('edge', 43), ('love', 43), ('year', 42), ('seller', 40), ('sim', 39), (' ve', 39), ('carrier', 37), ('glass', 36), ('picture', 35), ('look', 34), ('buy', 34), ('pay', 33), ('charger', 33), ('s6', 33), ('warranty', 32), ('service', 32), ('way', 32), ('iphone', 32), ('software', 31), ('charge', 31), ('get', 31), ('week', 30), ('review', 30), ('price', 29), ('amazon', 28), ('apps', 28), ('color', 28), ('’', 28), ('power', 27), ('hour', 26), ('purchase', 26), ('call', 26), ('people', 25), ('support', 25), ('speak

In [60]:
verb_dict=dict((i, verb_words.count(i)) for i in verb_words)
verb_dict = sorted(verb_dict.items(), key=operator.itemgetter(1),reverse=True)
print((verb_dict))

[(' ', 231), ('get', 81), ('n t', 77), ('love', 73), ('bought', 67), ('got', 66), ('using', 58), ('came', 57), ('make', 47), ('say', 44), ('working', 42), ('used', 41), ('buy', 41), ('received', 39), ('come', 35), ('want', 34), ('take', 34), ('amazing', 33), ('go', 32), ('know', 32), ('getting', 30), ('made', 30), ('took', 29), ('expected', 28), ('worked', 27), ('charging', 27), ('went', 26), ('tried', 25), ('purchased', 23), ('keep', 23), ('started', 22), ('recommend', 22), ('going', 20), ('unlocked', 20), ('see', 20), ('seems', 19), ('run', 19), ('think', 19), ('stopped', 18), ('samsung', 18), ('find', 18), ('put', 17), ('dropped', 17), ('look', 17), ('use', 17), ('said', 16), ('work', 16), ('set', 15), ('setting', 15), ('buying', 14), ('need', 14), ('give', 14), ('replace', 14), ('called', 13), ('turn', 13), ('thought', 13), ('s7', 13), ('sent', 13), ('loved', 12), ('damaged', 12), ('arrived', 12), ('looking', 12), ('believe', 12), ('told', 11), ('charged', 11), ('included', 11), ('

In [61]:
adj_dict=dict((i, adj_words.count(i)) for i in adj_words)
adj_dict = sorted(adj_dict.items(), key=operator.itemgetter(1),reverse=True)
print((adj_dict))

[(' ', 462), ('great', 235), ('good', 173), ('new', 98), ('u', 67), ('international', 61), ('excellent', 54), ('nice', 51), ('n t', 48), ('samsung', 47), ('last', 41), ('best', 41), ('much', 40), ('screen', 34), ('unlocked', 33), ('s7', 29), ('perfect', 29), ('old', 29), ('happy', 28), ('fine', 28), ('overall', 26), ('awesome', 26), ('sim', 25), ('bad', 23), ('sure', 23), ('android', 23), ('easy', 22), ('able', 22), ('little', 20), ('many', 20), ('update', 20), ('different', 19), ('free', 19), ('better', 19), ('second', 18), ('fast', 18), ('black', 18), ('   ', 18), ('low', 18), ('due', 17), ('big', 17), ('long', 16), ('upgrade', 16), ('small', 16), ('full', 15), ('original', 15), ('first', 15), ('right', 15), ('  ', 15), ('light', 15), (' s', 15), ('open', 14), ('high', 14), ('hot', 13), ('clear', 13), ('sd', 13), ('hard', 12), ('previous', 12), ('american', 12), ('large', 12), ('fit', 12), ('beautiful', 12), ('couple', 12), ('galaxy', 12), ('front', 12), ('extra', 11), ('fantastic', 

In [62]:
imp_nouns=[]
imp_verbs=[]
imp_adjs=[]
for k,v in noun_dict:
    if(v>=5):
        imp_nouns.append(k)
for k,v in verb_dict:
    if(v>=5):
        imp_verbs.append(k)
for k,v in adj_dict:
    if(v>=5):
        imp_adjs.append(k)
        

In [63]:
print(len(imp_nouns),len(imp_verbs),len(imp_adjs))

337 160 171


In [64]:
print(imp_nouns)

[' ', 'phone', 'samsung', 'work', ' s', 's7', 'camera', 'battery', 'time', 'screen', '  ', 'card', 'galaxy', 'problem', '   ', 'month', 'everything', 'version', 'day', 'use', 'issue', 'product', ' m', 'n t', 'thing', 'device', 'quality', 'life', 'case', 'box', 'feature', 'water', 'edge', 'love', 'year', 'seller', 'sim', ' ve', 'carrier', 'glass', 'picture', 'look', 'buy', 'pay', 'charger', 's6', 'warranty', 'service', 'way', 'iphone', 'software', 'charge', 'get', 'week', 'review', 'price', 'amazon', 'apps', 'color', '’', 'power', 'hour', 'purchase', 'call', 'people', 'support', 'speaker', 'display', 'lot', 'note', 'store', 'nothing', 'customer', 'change', 'hand', 'storage', 'app', 'gold', 's5', 'size', 'smartphone', 'speed', 'button', 'model', 'micro', 'brand', 'home', 's4', 'feel', 'processor', 'complaint', 'headphone', ' ll', 'reason', 'fact', 'slot', 'waterproof', 'something', 'item', 'help', 'video', 'star', 'android', 'memory', 'front', 'condition', 'difference', 'perfect', 'data'

In [20]:
#data_samples=preprocessed_docs

In [21]:
#data_samples=data['Review_Text'].tolist()

In [22]:
# Use tf-idf features for NMF.
print("Extracting tf-idf features for NMF...")
tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2,
                                   stop_words='english')

Extracting tf-idf features for NMF...


In [23]:
lemmatised_docs=[]
for i in preprocessed_docs:
    str=' '.join(i)
    lemmatised_docs.append(str)
    

In [24]:
lemmatised_docs[5]

'bought seller wireless circle u customer 4g lte marketfirst phone came uk electrical outlet fast charger outlet adapter u nt problem bulkiness expectedanother thing central midwest hard time anywhere received good signal strength 4g lte even called attempted new sim card thinking possibly card unlocked certain frequency band ordinarily would nt care except battery statistic constantly say cell standby using batterythis weekend real problem started using phone looked back second screen frozen tried end software everything unresponsive touch screen button tried restarting phone power button produced response older model would remove battery point time last 2 day tried charging performing various button press combination even taking device authorized retailer phone dead 10 month 550i would recommend buying phone unless solid plan returns/repairs/warranties'

In [25]:
print(type(data_samples[1]),type(lemmatised_docs[1]))

<class 'str'> <class 'str'>


In [26]:
tfidf = tfidf_vectorizer.fit_transform(lemmatised_docs)

In [27]:
tfidf.shape

(820, 1516)

In [28]:
tfidf_features=tfidf_vectorizer.get_feature_names()
len(tfidf_features)

1516

In [29]:
tfidf_features[:20]

['10',
 '100',
 '1000',
 '11',
 '12',
 '128',
 '12mp',
 '13',
 '14',
 '15',
 '16',
 '17',
 '1st',
 '20',
 '200',
 '200gb',
 '2016',
 '24',
 '29',
 '2k']

In [30]:
print(tfidf.shape)
# shape of the tfidf is 26,105 means 26 docs(reviews) and 105 features(words) with max_df=0.90, min_df=2

(820, 1516)


In [31]:
tfidf.getrow(0)

<1x1516 sparse matrix of type '<class 'numpy.float64'>'
	with 76 stored elements in Compressed Sparse Row format>

https://buhrmann.github.io/tfidf-analysis.html

In [32]:
def top_tfidf_feats(row, features,top_n):
    #top_n=25
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

In [33]:
def top_feats_in_doc(Xtr, features, row_id,top_n):
    #top_n=25
    ''' Top tfidf features in specific document (matrix row) '''
    row = np.squeeze(Xtr[row_id].toarray())
    return top_tfidf_feats(row, features,top_n)

In [34]:
def top_mean_feats(Xtr, features,top_n):
    grp_ids=None
    min_tfidf=0.1
    ''' Return the top n features that on average are most important amongst documents in rows
        indentified by indices in grp_ids. '''
    if grp_ids:
        D = Xtr[grp_ids].toarray()
    else:
        D = Xtr.toarray()

    D[D < min_tfidf] = 0
    tfidf_means = np.mean(D, axis=0)
    return top_tfidf_feats(tfidf_means, features,top_n)

In [35]:
def top_feats_by_class(Xtr, y, features,top_n):
    min_tfidf=0.1
    
    ''' Return a list of dfs, where each df holds top_n features and their mean tfidf value
        calculated across documents with the same class label. '''
    dfs = []
    labels = np.unique(y)
    for label in labels:
        ids = np.where(y==label)
        feats_df = top_mean_feats(Xtr, features)
        feats_df.label = label
        dfs.append(feats_df)
    return dfs

In [36]:
top_feats_in_doc(tfidf,tfidf_features,5,5)

,feature,tfidf
0,button,0.281664
1,tried,0.254092
2,outlet,0.219148
3,4g,0.208496
4,lte,0.204135


In [37]:
top_mean_feats(tfidf,tfidf_features,50)

,feature,tfidf
0,phone,0.080238
1,great,0.053702
2,good,0.049739
3,love,0.039986
4,work,0.033812
5,excellent,0.032260
6,product,0.025473
7,nt,0.024061
8,perfect,0.023918
9,samsung,0.020634


In [38]:
# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                               stop_words='english')

Extracting tf features for LDA...


In [39]:
tf = tf_vectorizer.fit_transform(lemmatised_docs)

In [40]:
print(tf_vectorizer.vocabulary)

None


In [41]:
tf_features=tf_vectorizer.get_feature_names()
len(tf_features)

1516

In [42]:
tf_vectorizer.get_feature_names()[:20]

['10',
 '100',
 '1000',
 '11',
 '12',
 '128',
 '12mp',
 '13',
 '14',
 '15',
 '16',
 '17',
 '1st',
 '20',
 '200',
 '200gb',
 '2016',
 '24',
 '29',
 '2k']

In [43]:
print(tf.shape)

(820, 1516)


In [44]:
tf.data

array([1, 1, 1, ..., 1, 3, 2], dtype=int64)

In [45]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [46]:
# Fit the NMF model
print("Fitting the NMF model (Frobenius norm) with tf-idf features, "
      "n_samples=%d and n_features=%d..."
      % (1000, 1000))
nmf = NMF( random_state=1,
          alpha=.1, l1_ratio=.5).fit(tfidf)

Fitting the NMF model (Frobenius norm) with tf-idf features, n_samples=1000 and n_features=1000...


In [47]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

In [ ]:
print("\nTopics in NMF model (Frobenius norm):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, 50)


Topics in NMF model (Frobenius norm):
Topic #0: zoom floor fit fitting fix fixed flagship flash flat flaw flawlessly fly firm focus focusing folk followed following foot forced fragile frame firmware finish fantastic feel fari fast faster fastest fault faulty favorite feature fee fell finicky felt figure file filled finally finding fine finger fingerprint free freeze freezing glad
Topic #1: zoom floor fit fitting fix fixed flagship flash flat flaw flawlessly fly firm focus focusing folk followed following foot forced fragile frame firmware finish fantastic feel fari fast faster fastest fault faulty favorite feature fee fell finicky felt figure file filled finally finding fine finger fingerprint free freeze freezing glad
Topic #2: zoom floor fit fitting fix fixed flagship flash flat flaw flawlessly fly firm focus focusing folk followed following foot forced fragile frame firmware finish fantastic feel fari fast faster fastest fault faulty favorite feature fee fell finicky felt figure f

Topic #1187: zoom floor fit fitting fix fixed flagship flash flat flaw flawlessly fly firm focus focusing folk followed following foot forced fragile frame firmware finish fantastic feel fari fast faster fastest fault faulty favorite feature fee fell finicky felt figure file filled finally finding fine finger fingerprint free freeze freezing glad
Topic #1188: zoom floor fit fitting fix fixed flagship flash flat flaw flawlessly fly firm focus focusing folk followed following foot forced fragile frame firmware finish fantastic feel fari fast faster fastest fault faulty favorite feature fee fell finicky felt figure file filled finally finding fine finger fingerprint free freeze freezing glad
Topic #1189: zoom floor fit fitting fix fixed flagship flash flat flaw flawlessly fly firm focus focusing folk followed following foot forced fragile frame firmware finish fantastic feel fari fast faster fastest fault faulty favorite feature fee fell finicky felt figure file filled finally finding fin

In [ ]:
print("Fitting the NMF model (generalized Kullback-Leibler divergence) with "
      "tf-idf features, n_samples=%d and n_features=%d..."
      % (1000, 1000))
nmf = NMF( random_state=1,
          beta_loss='kullback-leibler', solver='mu', max_iter=1000, alpha=.1,
          l1_ratio=.5).fit(tfidf)

Fitting the NMF model (generalized Kullback-Leibler divergence) with tf-idf features, n_samples=1000 and n_features=1000...


In [ ]:
print("\nTopics in NMF model (generalized Kullback-Leibler divergence):")
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, 10)

In [ ]:
print("Fitting LDA models with tf features, "
      "n_samples=%d and n_features=%d..."
      % (1000, 1000))
lda = LatentDirichletAllocation(max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [ ]:
lda.fit(tf)

In [ ]:
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, 20)

In [ ]:
# Create training and test sets
train, test = train_test_split(data['reviewText'],test_size=0.2,random_state=1)

In [ ]:
test.shape

In [ ]:

vect = CountVectorizer().fit(train)
vect.get_feature_names()[::5000]

In [ ]:

len(vect.get_feature_names())

In [ ]:
# transform the documents in the training data to a document-term matrix
train_vectorized = vect.transform(train)

train_vectorized

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Fit the TfidfVectorizer to the training data specifiying a minimum document frequency of 5
tfidfvect = TfidfVectorizer(min_df=10).fit(train)
len(tfidfvect.get_feature_names())

In [ ]:
tfidfvect.get_feature_names()[10000:100100]